Github repository link: https://github.com/Sumedh220122/Fine-tunining-IBM-granite-to-interpret-heat-equation

In [1]:
import os

In [2]:
!pip install transformers==4.46.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.6 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [3]:
import transformers
print(transformers.__version__)

4.46.0


1, IMPORTING MODULES AND LOADING THE MODEL

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
import pandas as pd
import json
from datasets import Dataset
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [5]:
device = "auto"
model_path = "ibm-granite/granite-3.1-2b-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)

tokenizer_config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [7]:
print(model)

GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49152, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): GraniteRMSNorm((20

In [8]:
def tokenize_function(examples):
    text = examples["prompt"]

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

2. DATA LOADING AND PREPROCESSING

In [9]:
# A data Collator ensures that my inputs are paddded properly with the required amount durin training to
# ensure uniformity between token lengths

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
#The following function is used to create a json file for the entire combined data
def create_jsonl_from_df(df, output_file):
    with open(output_file, 'w') as f:
        for _, row in df.iterrows():
            qa_pair = {
                "prompt": row["Questions"], 
                "completion": row["Answers"]
            }
            json.dump(qa_pair, f)
            f.write("\n")

In [11]:
df1 = pd.read_csv('/kaggle/input/heat-solution-dataset/case1.csv').sample(n=500, random_state=42)
df2 = pd.read_csv('/kaggle/input/heat-solution-dataset/case2.csv').sample(n=500, random_state=42)
df3 = pd.read_csv('/kaggle/input/heat-solution-dataset/case3.csv')
df4 = pd.read_csv('/kaggle/input/heat-solution-dataset/case4.csv')

full_df = pd.concat([df1, df2, df3, df4], ignore_index = True)

create_jsonl_from_df(full_df, 'qa_pairs.jsonl')

3. MODEL TRAINING

In [12]:
# Splitting the data into train and val pairs for training

dataset = Dataset.from_json('qa_pairs.jsonl')

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

tokenized_train = train_dataset.map(tokenize_function, batched = True)
tokenized_val = val_dataset.map(tokenize_function, batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [13]:
# PEFT is a fine-tuning method used in LLMS. Peft helps us to train only few of the parameters of the 
# model while freezing the rest. This substantially reduces memory requirements and training time.

peft_config = LoraConfig(task_type="CAUSAL_LM",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_proj', 'k_proj', 'v_proj'])

In [40]:
# The cosine similarity is the metric used in training i.e fine-tuning the LLM

def compute_metrics(p):
    preds, labels = p

    preds = preds.cpu()
    labels = labels.cpu()
    
    generated_answers = [tokenizer.decode(pred, skip_special_tokens=True) for pred in preds]
    reference_answers = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
    cosine_similarities = [compute_cosine_similarity(gen, ref, embedder) for gen, ref in zip(generated_answers, reference_answers)]
    
    avg_cosine_similarity = sum(cosine_similarities) / len(cosine_similarities)
    
    return {'cosine_similarity': avg_cosine_similarity}

In [15]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,474,560 || all params: 2,535,000,064 || trainable%: 0.0582


In [16]:
lr = 1e-3  # Learning rate
num_epochs = 10
batch_size = 4

training_args = TrainingArguments(
    output_dir= '/kaggle/working/model_output',
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_steps = 1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cuda'

In [41]:
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-41-49deba5a57a4>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [20]:
model.save_pretrained("modified_granite_model")
tokenizer.save_pretrained("modified_granite_model")

('modified_granite_model/tokenizer_config.json',
 'modified_granite_model/special_tokens_map.json',
 'modified_granite_model/vocab.json',
 'modified_granite_model/merges.txt',
 'modified_granite_model/added_tokens.json',
 'modified_granite_model/tokenizer.json')


4. PERFORMING INFERENCE

In [21]:
def load_model():
    device = "auto"
    model_path = "modified_granite_model"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)
    model.eval()
    return model, tokenizer

In [29]:
def get_model_response(model, tokenizer, prompt, max_tokens=300):
    input_text = f"Question: {prompt}\n\nAnswer:"
    input_tokens = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    output = model.generate(
        **input_tokens,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode and clean up the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    # Remove the input prompt from the response
    response = response[len(input_text):].strip()
    return response

In [30]:
def model_inference():
    model, tokenizer = load_model()
    
    cases = {
        "Case1Q1": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what is the temperature distribution at the corner (0, 0) of the unit square mesh?""",
        "Case1Q2": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        how does the temperature change with respect to the position along the x-axis at y = 0.5?""",
        "Case1Q3": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        if we increase the coeeficient of pi in the force function what will happen?""",
        "Case2Q1": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        explain why the temperature is zero at both x=0 and x=1, and what this means physically.""",
        "Case2Q2": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        at what coordinates does the maximum temperature occur, and what determines this location?""",
        "Case2Q3": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        how does the temperature profile change along the vertical line x=0.5 compared to x=0.25?""",
        "Case3Q1": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what is the temperature at the corner (0, 0) of the unit square mesh?""",
        "Case3Q2": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what physical significance does the boundary condition u(0,y)=0 have in the context of heat diffusion on the unit square mesh?""",
        "Case3Q3": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what does the boundary condition u(1,y)=y(1−y) represent physically in this heat diffusion problem?""",
        "Case4Q1": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        what can you infer about the decay rate of temperature!""",
        "Case4Q2": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        Comment on the physical interpretation of why the spatial pattern remains unchanged while only the amplitude decreases with time@""",
        "Case4Q3": """Analyze this steady-state heat equation solution T(x,y) = x^2+y^2 in a unit square domain and tell 
        What is the effect of alpha on the deacy rate of heat dissipation#"""
    }
    
    results = []
    
    for case, prompt in cases.items():
        print(f"\nTesting {case}...")
        try:
            response = get_model_response(model, tokenizer, prompt)
            results.append(response)
            print(f"\nResponse for {case}:")
            print(response)
        
        except Exception as e:
            print(f"Error in {case}: {str(e)}")
            results.append(f"Error: {str(e)}")
    
    return results

In [31]:
results = model_inference()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Testing Case1Q1...

Response for Case1Q1:
If you look at the boundary conditions, you have a Dirichlet boundary condition at the boundary 
        of the domain.  The value of the function T(x,y) at the boundary is zero.  So, we will use the fact 
        that at the corner of the domain (0,0), the boundary condition is zero and plug in the solution.

#%%
import numpy as np
import matplotlib.pyplot as plt

#%%
T = lambda x, y: x**2+y**2
T_mesh = np.zeros((50, 50))
for i in range(50):
    for j in range(50):
        T_mesh[i, j] = T(i/50, j/50)

#%%
plt.imshow(T_mesh, cmap=plt.cm.seismic)
plt.colorbar()
plt.show()

#%%
T_corner = T(0,0)
print(T_corner)

#%%
plt.imshow(T_mesh, cmap=plt.cm.seismic)
plt.colorbar()
plt.show()

#%%

#%%

#%%

Testing Case1Q2...

Response for Case1Q2:
T(x,y) = x^2+y^2. If we put y=0.5, then T(x,y) = x^2 + 0.5^2. Since x is a function of y, we can 
        solve for x by writing y = 0.5 in terms of x, i.e., x = sqrt(0.5^2 - y^2), and then substitute this 
   

In [32]:
df = pd.DataFrame({"Id" : list(range(1,13)), "Answer": results})
df.to_csv('submission.csv',index = False)